In [0]:
# set up the notebook and download the data
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME =ahmedaraby
%env KAGGLE_KEY =af9b0168bb30bb0c4ec617dc6e25c015
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip "fcis-sc-deeplearning-competition.zip"

In [0]:
# imports 

import pickle
import os 
from os import listdir
import numpy as np
import pandas as pd  
import cv2 
import matplotlib.pyplot as plt  
import matplotlib.image as mpimg

# tensorflow 
import tensorflow as tf 
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization , batch_normalization 
from tflearn.optimizers import Momentum
from tflearn.layers.estimator import regression
from tflearn.optimizers import SGD
from tflearn.initializations import normal , xavier
from __future__ import division, print_function, absolute_import

# keras 
import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.layers import Conv2D , MaxPooling2D
from keras.models import load_model
from keras.regularizers import l2
from keras.optimizers import SGD , Adam
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


In [0]:
#### utils 


def save_into_dics(obj , name):
    # name should include the extension
    pickle_out = open(name , 'wb')
    pickle.dump(obj , pickle_out)
    pickle_out.close()
    return


def read_from_dics(name):
    pickle_in = open(name , 'rb')
    obj = pickle.load(pickle_in)
    pickle_in.close()
    return obj


def shuffle(X , Y):
    index = np.random.permutation(X.shape[0])

    X = X[index]
    Y = Y[index]

    return X , Y



In [0]:
# preprocessing 


def global_mean_per_channel(X):

    mean= np.mean(X , axis = (0 , 1 , 2))   # it divides by (number of examples * rows * cols )

    print("mean of the whole data set is per channel " , mean)
    np.save("DS_local_mean.npy" , mean)
    return mean

def normalize_min_max(img):
    min =  np.min(img)
    max = np.max(img)
    img = (img - min) / (max-min)  
    return img

def resize_image(img , inputH = 256 , inputW = 256 , inputC = 3):
    H , W , _ = img.shape

    # resize the shorter side
    # handle very small images 
    toH = inputH 
    toW = inputW

    if W >= H and W>=inputH:
        toW = W
    if H >= W and H>=inputW:
        toH = H

    resized_img = cv2.resize(img, dsize=(toH, toW))
    # resized_img = resized_img.reshape(toH , toW , inputC)  # big massive bug   **********************************************

    # get the start/ end points of the center 256*256
    H , W  , _ = resized_img.shape

    midH = int(H/2)
    midW = int(W/2)

    # the % thing is to handle non even windows 
    startH = midH - inputH//2
    endH = midH + inputH//2 + (inputH%2)  # excluded

    startW  = midW - inputW//2
    endW = midW + inputW//2 + (inputW%2)

    # crop center inputH * inputW
    croped_img = resized_img[startH:endH , startW:endW , :]

    H , W , C = croped_img.shape
    assert (H==inputH and W == inputW and C== inputC)
    return croped_img


def preprocessing(img  , inputH=256 , inputW=256 , inputC=3):
    img = resize_image(img , inputH , inputW , inputC)
    return img

def dataset_preprocessing(X , mode=1):

    # centering then normalization [0 , 1]
    if mode==1:
        print("centering then normalization ") 
        #mean = global_mean_per_channel(X)
        mean = np.load("DS_local_mean.npy")
        #mean = np.array(mean).reshape(1, 1, 1, 3)  
        X = X - mean

        # normalization
        for i in range(0 , X.shape[0], 1):
            X[i] = normalize_min_max(X[i])


    #[ TO DO ]
    # normalization then centering
    else :
      assert("not implemented yet")

    return X

In [0]:
##### agumentation 
import imutils  # for rotating without cropping the edges
# for preprocessing  
inputH = 256
inputW = 256
inputC = 3

# to be feed for the CNN
CNNH = 227
CNNW = 227
CNNC = 3

mode = -1 # -1 BGR , 0  gray

def horizontal_reflication_img(img , CNNH=227 , CNNW=227 , CNNC=3):
    H, W, C = img.shape
    assert (H == CNNH and W == CNNW and C == CNNC)
    hflip = cv2.flip(img , 1)
    return hflip

def extract_random_portion(img , CNNH = 227, CNNW = 227, CNNC = 3):
    H , W , C = img.shape
    assert (H==inputH and W==inputW and C==inputC)

    startH = np.random.randint(low=0 , high=H - CNNH)
    startW = np.random.randint(low=0 , high=W - CNNW)
    endH = startH + CNNH
    endW = startW + CNNW

    assert (endH <= H and endW <=W)
    ex_img = img[startH:endH , startW:endW , :]
    H , W , C = ex_img.shape
    assert (H == CNNH and W == CNNW and C == CNNC)
    return ex_img

def rotate_img_keep(img):
    # images get reshaped
    angle = np.random.random_integers(low=-45 , high=45)
    rot_img = imutils.rotate_bound(img , angle)
    return rot_img

def add_gauusian_noise(img):
    blured_img = cv2.GaussianBlur(img , (7 , 7) , 1.5)
    return blured_img

def add_salt_peper_noise(img):
    salty_img = np.copy(img)
    for i in range(0 , img.shape[0]):
        for j in range(0 ,  img.shape[1]):
            rand = np.random.random_sample()
            if rand <= 0.03:
                salty_img[i , j , :] = 0
            elif rand <= 0.07:
                salty_img[i, j, :] = 255
    return salty_img

def Agumentation(img, CNNH=227 , CNNW=227 , CNNC=3):
    ex_img = extract_random_portion(img , CNNH , CNNW , CNNC)
    Hfliped_img = horizontal_reflication_img(ex_img , CNNH , CNNW , CNNC) 

    rotate_img = rotate_img_keep(ex_img)   #rotate change the size of the image
    gauss_img = add_gauusian_noise(ex_img)
    #salty_img = add_salt_peper_noise(ex_img)
    # make sure they are 227
    rotate_img = cv2.resize(img, dsize=(227, 227))
    # string is to make the image unique form the original
    img_l = [ ( ex_img  , "227"), ( Hfliped_img , "flip"), (rotate_img , "rotate") , (gauss_img , "gauss") ] #, (salty_img , "salty")]
    # string is to make the image unique form the original

    #img_l = [ ( ex_img  , str(CNNH)), ( Hfliped_img , "flip")]
    return img_l

In [0]:
# generate train and validation data 

# labels as in kaggle but 0 - based 
label_dict = {'airport_inside': 0, 'bakery': 1, 'bedroom': 2, 'greenhouse': 3,
              'gym': 4, 'kitchen': 5, 'operating_room': 6, 'poolinside': 7,
              'restaurant': 8, 'toystore': 9}

# for preprocessing  
inputH = 256
inputW = 256
inputC = 3

# to be feed for the CNN
CNNH = 227
CNNW = 227
CNNC = 3

mode = -1 # -1 BGR , 0  gray

# image.shape (height ,  width , channels )

SLASH = '/'
corrupted = ['indooPool_Inside_gif.jpg']

def get_path_list(path):
    path_list = listdir(path)
    return path_list

def check_image(img):
    if mode==-1 and ( len(img.shape)!=3 or img.shape[2] != 3): # img.shape[0] < 256 or img.shape[1] < 256 or (we will do upsampling)
        return 0
    return 1

def split_data(path):
    # file with 10 folders
    file_list =  get_path_list(path)

    X_train = []
    Y_train = []
    X_valid = []
    Y_valid = []
    cnt = 0

    for i in range(0 , len(file_list) , 1):
        # specific file of the 10
        class_path = path + SLASH + file_list[i]

        # images paths in 1 folder of the 10 folders
        class_path_list = get_path_list(class_path)

        # iterate throw the images of 1 class
        tl_img   =   []
        tl_label =   []
        end = len(class_path_list) - int(0.1*len(class_path_list))
        for j in range(0 , len(class_path_list) , 1):
            final_path = class_path + SLASH + class_path_list[j]

            # corrupted images
            if class_path_list[j] in corrupted:
                continue

            img = cv2.imread(final_path , mode) 

            if check_image(img)==0:
                continue

            print("final_path : "  , final_path)
            #print("class name " , file_list[i])

            img = img[: , : , (2 , 1 , 0)]    # converting the image to be RGB 
            cnt+=1
            # apply preprocessing and agumentation
            img = preprocessing(img , inputH , inputW , inputC)
            img_l = Agumentation(img , CNNH , CNNW , CNNC)  

            for q in range(0 , len(img_l)):
                timg = img_l[q][0]
                tl_img.append(timg)
                tl_label.append(label_dict[file_list[i]])

        # balance one class beteen train and valid   data ...
        index = np.random.permutation(len(tl_img))
        end = len(tl_img) - int(0.1 * len(tl_img))

        for q in range(0 , len(index)):
          ind = index[q]
          if q < end:            
            X_train.append(tl_img[ind])
            Y_train.append(tl_label[ind])
          else:
            X_valid.append(tl_img[ind])
            Y_valid.append(tl_label[ind])

    
    # reshape the data
    X_train = np.array(X_train).reshape(-1 , CNNH , CNNW , CNNC)
    Y_train = np.array(Y_train).reshape(-1, 1)

    X_valid = np.array(X_valid).reshape(-1, CNNH, CNNW, CNNC)
    Y_valid = np.array(Y_valid).reshape(-1, 1)
    
    # final preprocessing 
    #X_train = dataset_preprocessing(X_train , 1)
    #  *************************

    # shuffle 
    X_train , Y_train = shuffle(X_train ,Y_train)
    
    print("count " , cnt)
    print("train data shape " , X_train.shape , Y_train.shape)
    print("valid data shape " , X_valid.shape , Y_valid.shape)
    


    # cast 
    X_train = X_train.astype(np.float32)
    

    # save the data into the disc
    np.save("X_train.npy", X_train)
    np.save("Y_train.npy", Y_train)

    np.save("X_valid.npy" , X_valid)
    np.save("Y_valid.npy" , Y_valid)

    return 
    
"""
def save_to_csv(image_path, labelid,name='train.csv'):
    tmp_dict = {"image_path":image_path , "labelid":labelid}
    Data = pd.DataFrame(tmp_dict)
    Data.to_csv(name)
    return Data
"""

print("start")
split_data("train/train")
print("end")

In [0]:

def VGG_16(lmda=0.005 , drop=0.7):

    # winit = RandomNormal(mean=0.0 , stddev=0.01)

    model = Sequential()

    # first block of conv layers
    model.add(Conv2D(64 , (3 , 3) , padding='same' , activation='relu' , input_shape=(227 , 227 , 3) , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(64 , (3 , 3) , strides=(1 , 1) , padding='same'  , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # second block of conv layers
    model.add(Conv2D(128 , (3 , 3) , padding='same' , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # third block of conv layers
    model.add(Conv2D(256 , (3 , 3) , padding='same' , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(256 , (3 , 3) , padding='same' , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # forth block
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # fifth block
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # flatten
    model.add(Flatten())  # channel last

    # fully connected layers
    model.add(Dense(4096 , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Dropout(rate=drop))

    model.add(Dense(4096, activation='relu' , kernel_regularizer=l2(lmda)  ))
    model.add(Dropout(rate=drop))

    model.add(Dense(10, activation='softmax',  kernel_regularizer=l2(lmda)  ))

    return model

def freeze_layers(model):
    print("freze conv layers")
    index = [ 0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
    for i in range(0 , len(index)):
        model.layers[index[i]].trainable=False   # bug *****************************

    print("freze FC layers")
    index = [19, 21, 23]
    for i in range(0, len(index)-2):
      model.layers[index[i]].trainable = False    # bug ********************
      
    return model

# load trianable parameters 

def set_conv_weights(model, index , path="/content/drive/My"+" "+"Drive/vgg16_weights.npz"):
    print("initialize  some conv layers ")
    W = np.load(path)
    block = 1
    layer = 1

    for i in range(0 , len(index)):
        # reset the pattern , by detecting
        # the max pool layer and flatten layer
        if i!=0 and index[i]-index[i-1]>1:
            block += 1
            layer = 1

        weight = "conv" + str(block) + '_' + str(layer) + '_' + 'W'
        bias = "conv" + str(block) + '_' + str(layer) + '_' + 'b'

        # make sure that we are the right match
        X , Y =model.layers[index[i]].get_weights()
        X = X.shape
        Y = Y.shape
        XX = W[weight].shape
        YY = W[bias].shape
        #print(X , XX , Y , YY)
        assert(X == XX and Y == YY)

        #print("for debugining " , index[i] , weight , bias)
        model.layers[ index[i] ].set_weights([W[weight] , W[bias]])
        layer +=1


    return model


def set_fc_weights(model , index, path="/content/drive/My"+" "+"Drive/vgg16_weights.npz"):
    print(" initialize some FC layers ")
    W = np.load(path)

    layer = 6
    for i in range(0 , len(index)-1): # leave the output layer
        model.layers[index[i]].set_weights([W["fc"+str(layer)+"_W"] , W["fc"+str(layer)+"_b"]])
        layer += 1

    return model

def set_weights(model):
    # I think we don't need to return the model back as we edit what in it not assign to it 
    print("in set weights")
    # conv layers
    index = [0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15 , 16]
    model = set_conv_weights(model, index)

    # fulle connected layers
    index = [19, 21, 23]
    model = set_fc_weights(model , index)
    return model



In [0]:
# VGG-16   keras from scratch

print("start")
X_valid = np.load("X_valid.npy")
Y_valid = np.load("Y_valid.npy")

X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')

sess = tf.Session()
Y_train = sess.run(tf.one_hot(Y_train , depth=10))
Y_valid = sess.run(tf.one_hot(Y_valid , depth=10))
sess.close()

Y_train = np.squeeze(Y_train)
Y_valid = np.squeeze(Y_valid)

print(X_train.shape , Y_train.shape, type(X_train[0][0][0][0]) , X_train[0][0][0])
print(X_valid.shape , Y_valid.shape,  type(X_valid[0][0][0][0]),  X_valid[0][0][0])

# model = VGG_16()
model = load_model('full_model_19.h5')

model = freeze_layers(model)

# model = set_weights(model)

sgd = SGD(lr=0.001 , momentum=0.9)

# opt = Adam(lr=0.01)
model.compile(optimizer=sgd , loss="categorical_crossentropy" , metrics=["accuracy"])


print(model.summary())

img1 = X_train[14]
#img2 = X_valid[31]

plt.imshow(img1)
#plt.imshow(img2)


# callbacks 

best_val_acc = ModelCheckpoint('bestmodel.h5' , monitor='val_acc' , save_best_only=True , mode='max')

model.fit(x=X_train , y=Y_train , batch_size=256 , epochs=50 ,callbacks=[best_val_acc], validation_data=(X_valid , Y_valid))

print("save weights")
model.save_weights('full_weights_19.h5')

print("save model")
model.save('full_model_19.h5')
print("end")


start
(11269, 227, 227, 3) (11269, 10) <class 'numpy.float32'> [0.50387   0.5412651 0.5861068]
(1247, 227, 227, 3) (1247, 10) <class 'numpy.float32'> [0.644804  0.6981307 0.7554192]





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

freze conv layers
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 227, 227, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 227, 227, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 113, 113, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)           

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# predict 
def get_5_portions(img , inputH=227 , inputW=227):
    H , W , _ = img.shape

    # first portion
    img_l = []
    img_l.append(img[0:227 , 0:227 , :])

    # second portion
    img_l.append(img[0:227 , W-228:W-1 , :])  # order of the second index matters !!!

    # third portion
    img_l.append(img[H-228:H-1 , 0:227 , :])

    # 4th portion
    img_l.append(img[H-228:H-1 , W-228:W-1 , :])

    # get center portion
    midH = H//2
    midW = W//2

    startH = midH - inputH//2
    endH = midH + inputH//2 + (inputH%2)

    startW = midW - inputW//2
    endW = midW + inputW // 2 + (inputW % 2)
    img_l.append(img[startH:endH , startW:endW , :])

    # check
    """
    index = 0
    for timg in img_l:
        H , W , C = timg.shape
        # print(index)  # for debuging
        assert(H==inputH and W==inputW and C==3)
        index +=1
    """
    return img_l

def predict(X):
    labels = model.predict(X)
    labels = np.squeeze(np.argmax(labels , axis=1))

    labels = np.bincount(labels)
    labels= np.argmax(labels)
    return labels

def test(path):
    img_name  =  []
    img_label =  []

    mean = np.load('DS_local_mean.npy')
    mean = mean.reshape(1 , 1 , 3)

    file = listdir(path)
    for name in file:
        final_path = path + "/" + name #os.path.join(path , name)
        img = cv2.imread(final_path , -1)

        ###################################################################################

        # bad image
        if name=='1412_mb_file_0a8c5_gif.jpg':   # gif image
            print("bad image ")
            img_name.append(name)
            img_label.append(5)  # grean house 
            continue
        if name=="operating_room_07_06_altavista.jpg":  # gray scale img and my model take only RGB
            print("bad image ")
            img_name.append(name)
            img_label.append(3) # operating room 
            continue

        ####################################################################################

        # put the image in the same conditions of training
        print(name)
        # RGB
        img = img[: , : , (2 , 1 , 0)]

        # resize into 256 * 256
        img = preprocessing(img)

        # subtract the mean
        #img = img - mean

        # normalize the image
        #img = normalize_min_max(img)

        # this procedure is from alexnet - image net paper
        # generate all the agumented imgs and vote for label
        img_l = []
        img_l = get_5_portions(img) # 227*227
        #get the flips of each image
        sz = len(img_l)
        for i in range(0 , sz):
            fimg= horizontal_reflication_img(img_l[i])
            img_l.append(fimg)
        img_l = np.array(img_l).reshape(-1 , 227 , 227 , 3)
        label = predict(img_l)

        # build the file
        img_name.append(name)
        img_label.append(label+1)
    return img_name , img_label

def save_to_csv_file(names , labels):
  Data = {"id":names , "lable":labels}
  Data = pd.DataFrame(Data)
  Data.to_csv("mysubmit.csv")
  return Data

weight_path = "/content/drive/My"+" "+"Drive""/bestmodel3-96.h5"

print("start")
model = load_model("bestmodel.h5") #VGG_16()
#model.load_weights(weight_path)
names , labels = test('/content/test/test')
print("\n\n")
print(names)
print(labels)
print("end")


In [0]:

print("start")
# load data
X = np.load('/content/X.npy')
Y = np.load('/content/Y.npy')
sess = tf.Session()
Y = sess.run(tf.one_hot(Y , depth=10))
sess.close()
Y = np.squeeze(Y)

model = load_model("/content/drive/My"+" "+"Drive/bestmodel3-96.h5")
#print(model.summary())
#pred = model.predict(X)
print(model.evaluate(X , Y))

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory

model_file = drive.CreateFile({'title' : 'bestmodel_87.h5'})
model_file.SetContentFile('bestmodel_87.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})